# Notebook Summary

# Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.regularizers import l2
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
# from sklearn.preprocessing import StandardScaler
import datetime
%load_ext tensorboard
!rm -rf ./logs/

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# The LSTM Model Class

In [ ]:
class MyModel(Model):
  """
  LSTM Model

  Parameters
  ---------------------------
  num_classes: int
  Number of classes to predict

  Attributes
  ----------------------------
  lstm1: tf.keras.layers
  First layer of model

  lstm2: tf.keras.layers
  Second layer of model
  lstm3: tf.keras.layers
  Third layer of model

  dense_output: tf.keras.layers
  Output layer of model
  """
  def __init__(self, num_classes):
    super(MyModel, self).__init__()

    self.lstm1 = LSTM(units=512, return_sequences=True) # 64 is the units quantity. return_sequences must be True because we want return all the hidden states
    self.lstm2 = LSTM(256, dropout=0.4, return_sequences=True) 
    self.lstm3 = LSTM(128, dropout=0.4) 
    self.dropout = Dropout(0.4)
    self.dense_output = Dense(num_classes, activation='softmax')
  
  def call(self, inputs):
    """
    Call the layers and perform their operations on the input tensors
    :param inputs:  Input tensor
    :return:        Output tensor
    """
 
    x = self.lstm1(inputs)
    x = self.lstm2(x)
    x = self.lstm3(x)
    x = self.dropout(x)
    x = self.dense_output(x)

    return x  

In [ ]:
data = pd.read_csv('https://drive.google.com/uc?id=1uHzOe01O52giF-N8iw0e_biLziHkOcXU', encoding = 'utf-8', header=None)

In [ ]:
labels=pd.read_csv('https://drive.google.com/uc?id=1jN92fUr6FXp9DESStZM_rCynWAS-_DLA', encoding = 'utf-8', header=None)

# Preparing the Data

## Spliting the Data

In [ ]:
def split_data_subject(data, labels, subject): 
  # Get data from a subject, based on labels matrix.

  y_test_subj = labels[labels[1] == subject]   # get the samples of subject defined
  y_train_subj = labels[labels[1] != subject]  # get for train all the subjects except the subject for test

  X_train_subj = data.copy()
  X_train_subj = X_train_subj.drop(y_test_subj.index) # Delete from data test samples

  X_test_subj = data.copy()
  X_test_subj = X_test_subj.drop(y_train_subj.index)

  y_test_subj = y_test_subj.iloc[:,0:1].values
  y_train_subj = y_train_subj.iloc[:,0:1].values

  y_test_subj = y_test_subj - 1   # because initially our label is between 1 to 17, and we need put it to 0 to 16.
  y_train_subj = y_train_subj -1

  return X_train_subj, X_test_subj, y_train_subj, y_test_subj

In [ ]:
X_train_i28, X_test_i28, y_train_i28, y_test_i28 = split_data_subject(data, labels, 28) #Calling the function

# Transforming the Input Shape

In [ ]:
# Input shape: [samples, timesteps, feature]
# Reshape of 1d array to 3d array 
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
# Output will have 1 arrays that contains 4 arrays, each with 3 elements
newarr = arr.reshape(1, 4, 3)


In [ ]:
def change_shape(X_i, y_i):
  #Adapting the data for training

  X = X_i.copy().values
  # print(X[0,0], X[0,151], X[0,302])

  X_x = X[:, 0:151]
  X_y = X[:, 151:302]
  X_z = X[:, 302:453]

  # dstack: stack arrays in sequence depth wise (along third axis) 
  X_new = np.dstack([X_x, X_y, X_z]) 

  encoded = tf.keras.utils.to_categorical(y_i) # one-hot-encoded

  return X_new, encoded

# Working with the 28th subject

In [ ]:
X_new_train_i28, encodedtrain_i28 = change_shape(X_train_i28, y_train_i28)

In [ ]:
X_new_test_i28, encodedtest_i28 = change_shape(X_test_i28, y_test_i28)

# Classifying UniMiB-SHAR with a LSTM network

In [ ]:
#!rm -rf ./logs/

# Subject 28

# Setting Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #directory

callbacks = [tf.keras.callbacks.ModelCheckpoint('best', monitor= 'val_accuracy',
                                                verbose=1, save_best_only=True),
             tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1, 
                                            write_graph=True)]

num_classes = 17 # We have 17 kinds of activities in UNIMIB-CHAR
with tf.device('/device:GPU:0'):
  model = MyModel(num_classes) # Instatiating
  model.compile(optimizer= tf.keras.optimizers.RMSprop(), 
                loss= 'categorical_crossentropy', metrics=['accuracy'])

model.fit(X_new_train_i28, encodedtrain_i28, batch_size=512, epochs=42, validation_data=(X_new_test_i28, encodedtest_i28),verbose=1, callbacks=callbacks)

Epoch 1/42
 1/23 [>.............................] - ETA: 3s - loss: 2.8347 - accuracy: 0.0762WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
23/23 [==============================] - ETA: 0s - loss: 2.3026 - accuracy: 0.2215
Epoch 00001: val_accuracy improved from -inf to 0.31646, saving model to best
23/23 [==============================] - 9s 410ms/step - loss: 2.3026 - accuracy: 0.2215 - val_loss: 1.8550 - val_accuracy: 0.3165
Epoch 2/42
23/23 [==============================] - ETA: 0s - loss: 2.0738 - accuracy: 0.2596
Epoch 00002: val_accuracy did not improve from 0.31646
23/23 [==============================] - 9s 376ms/step - loss: 2.0738 - accuracy: 0.2596 - val_loss: 1.8417 - val_accuracy: 0.3070
Epoch 3/42
23/23 [==============================] - ETA: 

# Viewing the Results

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
predictions = model.predict(X_new_test_i28) # Checking the preditions

In [ ]:
print(predictions[55])
print(encodedtest_i28[55])

In [ ]:
print(np.argmax(predictions[55]))
print(np.argmax(encodedtest_i28[55]))

In [ ]:
y_ints = [y.argmax() for y in encodedtest_i28]
p_ints = [y.argmax() for y in predictions]

In [ ]:
print(y_ints)
print(p_ints)